In [67]:
import os
import sys
import numpy as np
import pandas as pd
import requests
from tqdm.notebook import tqdm as tqdm
from functools import partial
from pathlib import Path
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool 
import errno
import time

In [30]:
# HELPERS
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python > 2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
        
# HELPERS

In [31]:
with open('/Users/quanle/locca_revolve/input/test_urls.txt', 'r') as f:
    test_urls = f.readlines()
test_urls = [x.strip() for x in test_urls]
in_urls = test_urls

In [32]:
with open('/Users/quanle/locca_revolve/input/user_agents.txt', 'r') as f:
    user_agents = f.readlines()
user_agents = [x.strip() for x in user_agents]

In [33]:
url_prefix = 'https://chroniclingamerica.loc.gov/data/batches/'
batch_names = [x.replace(url_prefix, '').split('/')[0] for x in test_urls]
url_tails = [x.replace(url_prefix, '').split('/')[1:] for x in test_urls]
url_tails = ['/'.join(x) for x in url_tails]

out_path = "./scans/"
out_files = [x.replace(url_prefix, out_path) for x in test_urls]

In [34]:
in_zip = list(zip(in_urls, out_files))

In [72]:
my_chunks = list(chunks(in_zip, 10))

In [73]:
def parallelize_download(chunk):
    for i in range(len(chunk)):
        HEADERS = {
            'User-Agent': np.random.choice(user_agents)
        }
        out_folder = '/'.join(chunk[i][1].split('/')[:-1])
        mkdir_p(out_folder)
        r = requests.get(chunk[i][0], headers=HEADERS)
        with open(in_zip[i][1], 'wb') as f:
            f.write(r.content)
    return 0          

In [74]:
pool = ThreadPool(32)
time0 = time.time()
results = list(tqdm(pool.imap(parallelize_download, my_chunks)))
time1 = time.time()
print('Time elapsed: ', time1-time0)

0it [00:00, ?it/s]